In [ ]:
#Importing classes from the Jinja2 library to load and render templates.
from jinja2 import Environment, FileSystemLoader
import json
import requests

def page_link(a):
    if ' ' in a:
        return a.replace(' ', '_')
    else:
        return a

# Creating a custom class that will use the GitHub API to access the templates.
class GitHubLoader:
    def __init__(self, repo, branch='master', token=None):
        self.repo = repo
        self.branch = branch
        self.token = token
        self.base_url = f'https://raw.githubusercontent.com/{repo}/{branch}/'

    def get_source(self, environment, template):
        path = f'templates/{template}'
        url = self.base_url + path
        headers = {}
        if self.token:
            headers['Authorization'] = f'token {self.token}'
        response = requests.get(url, headers=headers)
        if response.status_code == 404:
            raise TemplateNotFound(template)
        elif response.status_code != 200:
            raise Exception(f"Failed to retrieve {url}")
        else:
            return response.text

# Creating an instance of the Environment class that uses the custom GitHubLoader.
environment = Environment(loader=GitHubLoader('harshulgupt/json_repo', token=PAT))
environment.globals['page_link'] = page_link
template = environment.get_template('homepage.html.j2')

json_files = ["General", "homepage"]
data = {}
for json_file in json_files:
    response = requests.get(
        f"https://api.github.com/repos/harshulgupt/json_repo/contents/jsons/{json_file}.json",
        headers={
            "Authorization": f"Token {PAT}"
        }
    )

    if response.status_code == 200:
        content = response.json()["content"]
        decoded_content = content.decode("utf-8")
        data[json_file] = json.loads(decoded_content)
content = template.render(General=data["General"], Homepage=data["homepage"])

# Open in write mode
with open('../index.html', mode='w', encoding='utf-8') as Homepage:
    Homepage.write(content)
